### MODELO Sistema Recomendación


### Librerias , carga de archivos

In [67]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors

In [68]:
movies_filt = pd.read_parquet('../datasets/movies_modelo.parquet')

In [89]:
# movies_filt = pd.read_parquet('movies_modelo.parquet')

Nuevo **Dataframe**: Filtro por pularidad

###  Preprocesamiento

- Features: genres, vote_average, popularity

In [88]:
type(movies_filt['genres'][0])

list

In [74]:
# Convertir columna 'genres' de una cadena de texto a una lista
movies_filt['genres'] = movies_filt['genres'].apply(lambda x: x.split(','))  # Asegúrate de que los géneros estén separados por comas


# Se codifica los géneros a binario
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(movies_filt['genres'])
genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_)

C:\Users\Angelica\AppData\Local\Temp\ipykernel_14580\2205104690.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_filt['genres'] = movies_filt['genres'].apply(lambda x: x.split(','))  # Asegúrate de que los géneros estén separados por comas


In [79]:
# Seleccionar los features numericos
features_num = movies_filt[['vote_average', 'popularity']]

# Se normaliza escalando
scaler = StandardScaler()
features_num_escaldas = scaler.fit_transform(features_num)
numeric_df = pd.DataFrame(features_num_escaldas, columns=features_num.columns)

In [80]:
# Se concatena los features normalizados y escalados
features_df = pd.concat([genres_df.reset_index(drop=True), numeric_df.reset_index(drop=True)], axis=1)


Similitud del coseno

In [81]:
# calculo de la matriz
cosine_simi = cosine_similarity(features_df, features_df)

### Funcion de recomnedación

In [95]:

def recomendacion(titulo, df):

    # Normalizar el título para la búsqueda
    titulo = titulo.lower()

    titulo_row = df[df["title"].str.lower() == titulo ]
    
    # Se verifica si la película existe en la base de datos
    if titulo_row.empty:
        return 0

    # Se obtiene el indice dado el titulo
    idx = titulo_row.index[0]

    # Obtengo los puntajes de similitud 
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Se ordena según la similitud de puntajes en orden descendente
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Indices de las películas similares
    sim_indices = [i[0] for i in sim_scores if i[0] != idx]

    # Se verifica si peliculas similares
    if len(sim_indices) == 0:
        return f"No se encontraron películas similares a '{titulo}'."

    # Obtengo los titulos de las 5 películas más similares
    top_movies = df['title'].iloc[sim_indices[:5]].values.tolist()

    return top_movies


In [97]:
titulo_pelicula = 'Toy Story'
recomendaciones = recomendacion(titulo_pelicula, movies_filt)


In [98]:
if recomendaciones == 0: 
    print("Error: Película {titulo_pelicula} no encontrada en el dataset.")
else:
    for indice, i in enumerate(recomendaciones):
        print(f"{indice+1}. {i}")

1. Jaws
2. Monsters, Inc.
3. In a Heartbeat
4. 300
5. The Nice Guys
